<a href="https://colab.research.google.com/github/ajayseetharam510/sample-rag-ai-workflow/blob/main/Sample_RAG_Pinecone_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This is a sample only - please use for educational purposes only
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
!pip install -qU bs4 tiktoken openai langchain pinecone-client[grpc]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.2/216.2 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 16.7 MB/s eta 0:00:00
ERROR: pip's dependen

In [ ]:
pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.4 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader("/content/drive/MyDrive/sampledata/info.txt", encoding = 'UTF-8')
docs = loader.load()
len(docs)

1

In [ ]:
docs[0]

Document(page_content='Active Directory has two forms of common security principals: user accounts. These accounts represent a physical entity that is a person. \nActive Directory groups may be organized in a hierarchy like a folder structure. \nUsers are assigned to Active Directory groups.Users that are assigned to the Active Directory group are members of the group. Usually users with similar profile attributes of Organization, Department, Role are assigned to \nthe same Active Directory Group. \n \nUsers have profile attributes such as \nfirst name, Last Name, Organization, Department, Role, Manager Name and are assigned one or moe Groups. \nIf a user has a manager i.e. a Manager Name that is not "None" then the user\'s manager will have additional group assignments. \nThe significant profile attributes to determine Active Directory group assignment are\nOrganization, Department, Role and whether the user is a manager of other users. \n\nFirst name Ajay, Last Name S in the Organiza

In [ ]:
print(docs[0].page_content)

Active Directory has two forms of common security principals: user accounts. These accounts represent a physical entity that is a person. 
Active Directory groups may be organized in a hierarchy like a folder structure. 
Users are assigned to Active Directory groups.Users that are assigned to the Active Directory group are members of the group. Usually users with similar profile attributes of Organization, Department, Role are assigned to 
the same Active Directory Group. 
 
Users have profile attributes such as 
first name, Last Name, Organization, Department, Role, Manager Name and are assigned one or moe Groups. 
If a user has a manager i.e. a Manager Name that is not "None" then the user's manager will have additional group assignments. 
The significant profile attributes to determine Active Directory group assignment are
Organization, Department, Role and whether the user is a manager of other users. 

First name Ajay, Last Name S in the Organization Presales and Department Soluti

In [ ]:
print(docs[0].metadata)

{'source': '/content/drive/MyDrive/hackathon/rtdocs/info.txt'}


In [ ]:
data = [{'text': subdoc.page_content, 'url': subdoc.metadata['source']} for subdoc in docs]
len(data)

1

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding('p50k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)


In [ ]:
from uuid import uuid4
from tqdm.auto import tqdm

chunks = []

for idx, record in enumerate(tqdm(data)):
    texts = text_splitter.split_text(record['text'])
    chunks.extend([{
        'id': str(uuid4()),
        'text': texts[i],
        'chunk': i,
        'url': record['url']
    } for i in range(len(texts))])

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
len(chunks)


6

In [ ]:
import os
from google.colab import userdata
from openai import OpenAI
client = OpenAI(
  api_key=(userdata.get('openai_key')),
)

In [ ]:


res = client.embeddings.create(
    input=[
        chunks[0]['text'], chunks[1]['text'], chunks[2]['text'], chunks[3]['text'],chunks[4]['text'],chunks[5]['text']
    ],model =  "text-embedding-3-small"
)


In [ ]:
len(res.data)

len(res.data[0].embedding)

1536

In [ ]:
import itertools
import numpy as np


combinations = list(itertools.combinations(range(6), 2))
for first,second in combinations:
  print(f'Similarity between {first} and {second}', np.dot(res.data[first].embedding, res.data[second].embedding))


Similarity between 0 and 1 0.5501921204935998
Similarity between 0 and 2 0.4068608960672012
Similarity between 0 and 3 0.35134112976986875
Similarity between 0 and 4 0.40319281176857596
Similarity between 0 and 5 0.414872934099639
Similarity between 1 and 2 0.8793317617456816
Similarity between 1 and 3 0.7212678406516668
Similarity between 1 and 4 0.7459316064079102
Similarity between 1 and 5 0.6053748689426093
Similarity between 2 and 3 0.7924748877364971
Similarity between 2 and 4 0.8430144407049331
Similarity between 2 and 5 0.6562407072164111
Similarity between 3 and 4 0.7654113610857907
Similarity between 3 and 5 0.5909740101455857
Similarity between 4 and 5 0.8090753009206211


In [ ]:
[np.linalg.norm(res.data[i].embedding) for i in range(6)]

[1.00000004790224,
 0.9999999575974682,
 0.9999999893367852,
 0.999999982682163,
 0.9999999878580798,
 1.0000000729211158]

In [ ]:
len(res.data[0].embedding)

1536

In [ ]:
import pinecone

index_name = 'gpt-4-langchain-docs'

# initialize connection to pinecone

from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
        api_key=userdata.get('pinecone_api_key')
    )



## check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes():
     if does not exist, create index
        pc.create_index(
            index_name,
            dimension=len(res.data[0].embedding),
            metric='dotproduct',
            spec=ServerlessSpec(cloud="aws",region="us-east-1")
  )

# view index stats
pc.describe_index(index_name)

{'dimension': 1536,
 'host': 'gpt-4-langchain-docs-ux6l4qv.svc.aped-4627-b74a.pinecone.io',
 'metric': 'dotproduct',
 'name': 'gpt-4-langchain-docs',
 'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
 'status': {'ready': True, 'state': 'Ready'}}

In [ ]:
from tqdm.auto import tqdm
import datetime
from time import sleep



batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(chunks), batch_size)):
    # find end of batch
    print ("In loop - ",i)
    i_end = min(len(chunks), i+batch_size)
    meta_batch = chunks[i:i_end]
    # get ids
    ids_batch = [x['id'] for x in meta_batch]
    # get texts to encode
    texts = [x['text'] for x in meta_batch]
    # create embeddings (try-except added to avoid RateLimitError)
    try:
        res = client.embeddings.create(input=texts, model="text-embedding-3-small")
    except:
        done = False
        while not done:
            print ("Sleeping...")
            sleep(5)
            print ("Trying again.. ")
            try:
                res = client.embeddings.create(input=texts, model="text-embedding-3-small")
                done = True
            except:
                pass
    embeds = [record.embedding for record in res.data]
    # cleanup metadata
    meta_batch = [{
        'text': x['text'],
        'chunk': x['chunk'],
        'url': x['url']
    } for x in meta_batch]
    print ("Preparing data to upsert")
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    print("Upserting to Pinecone")
    index = pc.Index(index_name)
    # Comment to avoid duplicating data for successive demos
    index.upsert(vectors=to_upsert)

  0%|          | 0/1 [00:00<?, ?it/s]

In loop -  0
Preparing data to upsert
Upserting to Pinecone


In [ ]:
import numpy as np
def get_ids_from_query(index, input_vector):
    print("searching pinecone...")
    results = index.query(vector=input_vector,
                          top_k=10000, include_values=False)
    ids = set()
    print(type(results))
    for result in results['matches']:
        ids.add(result['id'])
    return ids


def get_all_ids_from_index(index, num_dimensions, namespace=""):
    #print(index.describe_index_stats())
    num_vectors = index.describe_index_stats(
    )["total_vector_count"]#[namespace]['vector_count']
    all_ids = set()
    while len(all_ids) < num_vectors:
        print("Length of ids list is shorter than the number of total vectors...")
        input_vector = np.random.rand(num_dimensions).tolist()
        print("creating random vector...")
        ids = get_ids_from_query(index, input_vector)
        print("getting ids from a vector query...")
        all_ids.update(ids)
        print("updating ids set...")
        print(f"Collected {len(all_ids)} ids out of {num_vectors}.")

    return all_ids



all_ids = get_all_ids_from_index(index, num_dimensions=1536, namespace="")
print(list(all_ids))

# https://docs.pinecone.io/docs/manage-data
index.fetch(list(all_ids))

Length of ids list is shorter than the number of total vectors...
creating random vector...
searching pinecone...
<class 'pinecone.core.client.model.query_response.QueryResponse'>
getting ids from a vector query...
updating ids set...
Collected 12 ids out of 12.
['126e1d9d-19d9-46f1-b468-3147a72bcfe3', 'cd3ee70c-7966-424c-a809-6af92e4b3ab2', '8666ebaf-cc44-48fb-b654-862745b78b45', 'bface4f8-0774-4184-99ca-0927eccd0fa8', 'a3300c5c-40ef-4778-929a-fe98921d6d1c', '395053b1-b940-43a1-9704-460c2228b6c0', 'fe641434-bfb8-49f3-bb00-fed6984aafbb', '3d8dde85-b951-4364-951d-e48037c2ee17', '0fac2c8c-700c-4740-a69a-9e0147fe857d', '86ca3080-63f6-4bc2-a739-7f5c7c16284f', '457fe024-59f3-44c6-ac9f-5be2f0f455b0', '7d14fbd3-3b00-4ef5-8468-89d7855bbc55']


{'namespace': '',
 'usage': {'read_units': 2},
 'vectors': {'0fac2c8c-700c-4740-a69a-9e0147fe857d': {'id': '0fac2c8c-700c-4740-a69a-9e0147fe857d',
                                                      'metadata': {'chunk': 4.0,
                                                                   'text': 'First '
                                                                           'name '
                                                                           'Jim, '
                                                                           'Last '
                                                                           'Name '
                                                                           'P '
                                                                           'in '
                                                                           'the '
                                                                           'Organization '
                      

In [ ]:
# Run some samples to tune the model
query = "Summarize the CLARI_ADVANCED_USER_GROUP"

# Get embedding from OpenAI
res = client.embeddings.create(input=[query], model="text-embedding-3-small")

xq = res.data[0].embedding


# get relevant contexts (including the questions)
res = index.query(vector=xq, top_k=5, include_metadata=True)
res

{'matches': [{'id': '0fac2c8c-700c-4740-a69a-9e0147fe857d',
              'metadata': {'chunk': 4.0,
                           'text': 'First name Jim, Last Name P in the '
                                   'Organization Presales and Department '
                                   'Solution Engineering and Role Director '
                                   'with Manager Satish  is in the '
                                   'CLARI_USER_GROUP \n'
                                   'First name Doug, Last Name I in the '
                                   'Organization Presales and Department '
                                   'Solution Engineering and Role VP with '
                                   'Manager Todd is in the ALL_SE_GROUP \n'
                                   'First name Doug, Last Name I in the '
                                   'Organization Presales and Department '
                                   'Solution Engineering and Role VP with '
                      

In [ ]:
# get list of retrieved text
contexts = [item['metadata']['text'] for item in res['matches']]

augmented_query = "\n\n---\n\n".join(contexts)+"\n\n-----\n\n"+query
print(augmented_query)

First name Jim, Last Name P in the Organization Presales and Department Solution Engineering and Role Director with Manager Satish  is in the CLARI_USER_GROUP 
First name Doug, Last Name I in the Organization Presales and Department Solution Engineering and Role VP with Manager Todd is in the ALL_SE_GROUP 
First name Doug, Last Name I in the Organization Presales and Department Solution Engineering and Role VP with Manager Todd is in the CLARI_ADVANCED_USER_GROUP 
First name Doug, Last Name I in the Organization Presales and Department Solution Engineering and Role VP with Manager Todd is in the SALESFORCE_ADVANCED_USER_GROUP 
First name Ka, Last Name K in the Organization Presales and Department  and Role Operations with Manager Doug is in the CLARI_ADMIN_USER_GROUP

---

First name Jim, Last Name P in the Organization Presales and Department Solution Engineering and Role Director with Manager Satish  is in the CLARI_USER_GROUP 
First name Doug, Last Name I in the Organization Presale

In [ ]:

# system message to 'prime' the model
primer = """You are Q&A bot. A highly intelligent system that answers
user questions based on the information provided by the user above
each question. If the information can not be found in the information
provided by the user you truthfully say "No information available for this group".
"""

res= client.chat.completions.create(
    model="gpt-4",
    messages=[ {"role": "system", "content": primer},
        {"role": "user", "content": augmented_query}],
    stream=False
)
res

ChatCompletion(id='chatcmpl-9ekiidMmhv9RW51OeTvAozDlfrHO3', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The CLARI_ADVANCED_USER_GROUP includes users working in the Presales Organization and Solution Engineering Department. Specific users in this group including Doug in the role of VP with Manager Todd and Satish in the role of Senior Director also with Manager Doug.', role='assistant', function_call=None, tool_calls=None))], created=1719499740, model='gpt-4-0613', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=49, prompt_tokens=819, total_tokens=868))

In [ ]:
from IPython.display import Markdown

display(Markdown(res.choices[0].message.content))

The CLARI_ADVANCED_USER_GROUP includes users working in the Presales Organization and Solution Engineering Department. Specific users in this group including Doug in the role of VP with Manager Todd and Satish in the role of Senior Director also with Manager Doug.